# Code your own polarizability tensor workflow, part 1

This main goal of this notebook is to show how the ``Workflow`` class can be used to quickly implement of a workflow (that is, to define a queue of BigDFT jobs to be run in order to compute a quantity afterwards, as a post-processing procedure).

## Using the Workflow class

This is a three-step process:
- first, define a list of jobs so as to initialize the workflow,
- then run the workflow (all the jobs will be performed sequentially),
- finally, post-process the results of the calculations to get the quantity of interest, here the polarizability tensor.

### Define your worflow

Only the base classes of the ``MyBigDFT`` package are required to do that, and might also require the help of extra packages such as ``numpy`` (you should not need it yet, though).

In [1]:
from mybigdft import Workflow, InputParams, Posinp, Atom, Job
import numpy as np

# Main input variables:
# - use the optimized structure for the N2 molecule (found using the
#   LDA exchange-correlation functional)
atoms = [Atom('N', [0, 0, 0]), Atom('N', [0, 0, 1.0935])]
pos = Posinp(atoms, units="angstroem", boundary_conditions="free")
# - use some non-default input parameters to get high quality results
inp = InputParams({"dft": {"rmult": [7, 9], "hgrids": 0.35}})
# Electric field amplitude to be applied in each direction
ef_amplitude = 1.e-4

# Initialize the queue of jobs in order to be able to compute the 
# polarizability tensor
pt_queue = []
pt_wf = Workflow(queue=pt_queue)

### Run the jobs of your workflow

Nothing important to code here, as you might only want to change the value of the attributes to suit your needs. 

In [2]:
pt_wf.run(nmpi=6, nomp=3, force_run=True)

Be sure to remove the ``force_run`` argument when you are happy with the results of the workflow: this means that the jobs will not have to be run again if you decide to run the notebook again from the beginning.

### Post-process your results

Now that you ran the workflow, you can use the results to extract meaningful quantities. You should be able to do the post-processing without having to add another argument to the ``post_proc`` function (you may want to add some attributes to the ``Job`` instances before running the workflow).

In [3]:
def post_proc(workflow):
    # Use the logfile of the calculations run by the workflow to
    # extract the polarizability tensor. Hint: use the 'dipole' 
    # attribute of the logfile to extract the dipole of the
    pol_tensor = np.zeros((3, 3))
    return pol_tensor

In [4]:
# Run the post-processing of the workflow to compute
pol_tensor = post_proc(pt_wf)
print(pol_tensor)

In [5]:
# Make sure your results are correct
expected_pol_tensor = np.zeros((3, 3))
assert np.array_equiv(pol_tensor, expected_pol_tensor)